In [1]:
import torch
import torchvision

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

from torch import nn

from torch import optim

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

train_data = datasets.CIFAR10(
    root = "./data",
    train = True,
    transform = transform,
    download = True
)

test_data = datasets.CIFAR10(
    root = "./data", 
    train = False,
    transform = transform, 
    download  = True
)

batch_size = 4
trainloader = DataLoader(train_data, batch_size, shuffle = False, num_workers = 0)
testloader = DataLoader(test_data, batch_size, shuffle = False, num_workers = 0)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
dataiter = iter(testloader)
image, label = dataiter.next()
print(f"imput:[B, C, H, W] {image.size()}")
print(f"output: {label.size()}")

imput:[B, C, H, W] torch.Size([4, 3, 32, 32])
output: torch.Size([4])


In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_relu_maxpool_fc = nn.Sequential(
            nn.Conv2d(3, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d((2,2)),
            nn.Flatten(start_dim = 1),
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10),
        )
    def forward(self, x):
        return self.conv_relu_maxpool_fc(x)

net = Net()
print(net)
i = 0
for para in net.parameters():
    print(f"para{i}: {para.size()}")
    i = i + 1

Net(
  (conv_relu_maxpool_fc): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=400, out_features=120, bias=True)
    (8): ReLU()
    (9): Linear(in_features=120, out_features=84, bias=True)
    (10): ReLU()
    (11): Linear(in_features=84, out_features=10, bias=True)
  )
)
para0: torch.Size([6, 3, 5, 5])
para1: torch.Size([6])
para2: torch.Size([16, 6, 5, 5])
para3: torch.Size([16])
para4: torch.Size([120, 400])
para5: torch.Size([120])
para6: torch.Size([84, 120])
para7: torch.Size([84])
para8: torch.Size([10, 84])
para9: torch.Size([10])


In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device is {device}")

net = net.to(device)

device is cpu


In [8]:
print(f"trainloder共有{len(trainloader.dataset)}组数据")
print(f"trainloder的每个batch有{len(trainloader)}组数据")
dataiter = iter(trainloader)
input, output = dataiter.next()
print(f"input的size为：{input.size()}, output的size为：{output.size()}")

trainloder共有50000组数据
trainloder的每个batch有12500组数据
input的size为：torch.Size([4, 3, 32, 32]), output的size为：torch.Size([4])


In [9]:
def train(dataloder, model, optimizer, loss_fn):
    size = len(dataloder.dataset)
    sumloss = 0
    for batch, (X, y) in enumerate(dataloder):
        X = X.to(device)
        y = y.to(device)
        # X, y = X.to(device), y.to(device)
        pred = model.forward(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        sumloss += loss.item()
        if batch % 2000 == 1999:
            current = (batch + 1) * len(X)
            print(f"loss: {sumloss / 2000:>7f}  [{current:>5d}/{size:>5d}]")
            sumloss = 0
        
            
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Accuracy: {(100 * correct): 0.1f}%, Avg loss: {test_loss:.8f}")
    
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum=0.9)
epoch = 2
for i in range(epoch):
    print(f"Epoch{i + 1}\n--------------------------------")
    train(trainloader, net, optimizer, loss_fn)
    test(testloader, net, loss_fn)
print("Done")

Epoch1
--------------------------------
loss: 2.240343  [ 8000/50000]
loss: 1.924404  [16000/50000]
loss: 1.681116  [24000/50000]
loss: 1.560773  [32000/50000]
loss: 1.546951  [40000/50000]
loss: 1.489432  [48000/50000]
Accuracy:  46.7%, Avg loss: 1.45188936
Epoch2
--------------------------------
loss: 1.438400  [ 8000/50000]
loss: 1.413015  [16000/50000]
loss: 1.364118  [24000/50000]
loss: 1.319360  [32000/50000]
loss: 1.342344  [40000/50000]
loss: 1.292275  [48000/50000]
Accuracy:  53.6%, Avg loss: 1.29431398
Done


In [8]:
print(len(testloader))
print(len(testloader.dataset))

2500
10000


In [9]:
net.eval()

Net(
  (conv_relu_maxpool_fc): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=400, out_features=120, bias=True)
    (8): ReLU()
    (9): Linear(in_features=120, out_features=84, bias=True)
    (10): ReLU()
    (11): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [10]:
print(net)

Net(
  (conv_relu_maxpool_fc): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=400, out_features=120, bias=True)
    (8): ReLU()
    (9): Linear(in_features=120, out_features=84, bias=True)
    (10): ReLU()
    (11): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [11]:
a = torch.ones(3, 4, 5)
print(a)
print(len(a))

tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]])
3


In [12]:
for X,y in testloader:
    print(X.size(), y.size())
    break

torch.Size([4, 3, 32, 32]) torch.Size([4])


In [13]:
a = torch.tensor([[1, 2, 3, 4],[3, 4, 5, 6]])

In [14]:
a.size()

torch.Size([2, 4])

In [15]:
a.max(1)

torch.return_types.max(
values=tensor([4, 6]),
indices=tensor([3, 3]))

In [16]:
a = torch.tensor(1)
b = torch.tensor(1)
a.equal(b)

True